In [ ]:
import pygame
import random
import sys

# Initialize pygame
pygame.init()

# Game settings
WIDTH, HEIGHT = 800, 600
win = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("🚀 Galactic Defender")

# Load images
bg = pygame.image.load("outer-space-background.jpg")
bg = pygame.transform.scale(bg, (WIDTH, HEIGHT))

ship_img = pygame.image.load("spaceship.png")
ship_img = pygame.transform.scale(ship_img, (50, 40))

# Colors
WHITE = (255, 255, 255)
RED = (255, 0, 0)
YELLOW = (255, 255, 0)

# Spaceship
ship_x = WIDTH // 2
ship_y = HEIGHT - 60
ship_speed = 6
ship_width, ship_height = 50, 40

# Obstacles
obstacle_width = 40
obstacle_height = 20
obstacles = []
obstacle_speed = 5

# Bullets
bullets = []
bullet_speed = 8

clock = pygame.time.Clock()
font = pygame.font.SysFont("Consolas", 24)

# Draw all game objects
def draw(ship_x, obstacles, bullets, score):
    win.blit(bg, (0, 0))  # background
    win.blit(ship_img, (ship_x, ship_y))  # spaceship
    for ox, oy, color in obstacles:
        pygame.draw.rect(win, color, (ox, oy, obstacle_width, obstacle_height))
    for bx, by in bullets:
        pygame.draw.rect(win, YELLOW, (bx, by, 5, 10))  # laser bullets
    text = font.render(f"Score: {score}", True, WHITE)
    win.blit(text, (10, 10))
    pygame.display.update()

# Intro story
def story_screen():
    win.fill((0, 0, 0))
    lines = [
        "🌌 Galactic Defender 🌌",
        "",
        "Year 3077. Humanity is under threat.",
        "Alien drones and asteroid mines block your path.",
        "You must deliver the Quantum Core across the galaxy.",
        "",
        "🚀 Controls: Arrow Keys = Move, SPACE = Shoot",
        "",
        "Press ANY KEY to begin..."
    ]
    y = 50
    for line in lines:
        text = font.render(line, True, WHITE)
        win.blit(text, (40, y))
        y += 40
    pygame.display.update()
    waiting = True
    while waiting:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit(); sys.exit()
            if event.type == pygame.KEYDOWN:
                waiting = False

def main():
    global ship_x
    score = 0
    obstacles = []
    bullets = []

    story_screen()  # show intro

    run = True
    while run:
        clock.tick(30)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                pygame.quit()
                sys.exit()

        # Key presses
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT] and ship_x > 0:
            ship_x -= ship_speed
        if keys[pygame.K_RIGHT] and ship_x < WIDTH - ship_width:
            ship_x += ship_speed
        if keys[pygame.K_SPACE]:
            # Fire bullet from middle of spaceship
            bullets.append([ship_x + ship_width//2 - 2, ship_y])

        # Add obstacles
        if random.randint(1, 25) == 1:
            color = random.choice([RED, YELLOW])
            obstacles.append([random.randint(0, WIDTH-obstacle_width), 0, color])

        # Move obstacles
        for i in range(len(obstacles)):
            obstacles[i][1] += obstacle_speed

        # Move bullets
        bullets = [[bx, by - bullet_speed] for bx, by in bullets if by > 0]

        # Check bullet collisions with obstacles
        new_obstacles = []
        for ox, oy, color in obstacles:
            hit = False
            for b in bullets:
                bx, by = b
                if (bx < ox + obstacle_width and bx + 5 > ox and
                    by < oy + obstacle_height and by + 10 > oy):
                    hit = True
                    bullets.remove(b)
                    score += 10  # increase score for destroying obstacle
                    break
            if not hit:
                new_obstacles.append([ox, oy, color])
        obstacles = new_obstacles

        # Check collisions between spaceship and obstacles
        for ox, oy, color in obstacles:
            if (ship_x < ox + obstacle_width and ship_x + ship_width > ox and
                ship_y < oy + obstacle_height and ship_y + ship_height > oy):
                print("💥 GAME OVER! Final Score:", score)
                run = False

        draw(ship_x, obstacles, bullets, score)

main()
